## Restaurant Review Classification Model
*deep learning for natural language processing - Pytorch*

- Using torchtext module to classify a restaurant review to one of three classes ('Good', 'Ok', 'Bad') of `taste` label.
- reviews are from mango plate (web scraping with selenium)
- applied LSTM with below hyperparameter:

```
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
```



In [1]:
import torch
import torch.nn as nn
from torchtext import data
from torchtext import datasets
import random

In [2]:
%matplotlib inline

In [11]:
from torchtext.data import Field

TEXT = Field(sequential=True,
             batch_first=True,
             tokenize=str.split,
             use_vocab=True,
            include_lengths = True)  

LABEL = data.LabelField()

In [47]:
train, test = data.TabularDataset.splits(
                                        path='./data', 
                                        train='train_data_taste.csv',
                                        test='test_data_taste.csv', format='csv',
                                        fields=[('Text', TEXT),('Label', LABEL)]

)

In [48]:
pd.read_csv('train_data_taste.csv').iloc[5:10]

,맛있어요,good
5,짠단의 향연치킨와플 와플치킨 이거 치킨 별로일줄 알았는데 생각보다 의외의 맛남에 감...,ok
6,170305.맛 분위기 가성비 재방문 의사 있음 주문한 메뉴 2명 방문 ...,good
7,가격대는 조금 있지만 그에 맞는 고기질이랑 갈때마다 맛있게 먹어요,good
8,기억 안 나는데 메인 메뉴인 피자와 샐러드 먹었음. 누구를 데리고 가도 욕은 먹지 ...,ok
9,타코를 좋아해서 타코집을 방문하는데 역시나 맛있었습니다 요리가 식지않고 계속해서 먹...,good


In [49]:
len(train), len(test)

(1754, 439)

In [15]:
# build vocab for train/label (mapping words to integers)
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [16]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x129f85f28>, {'good': 0, 'ok': 1, 'bad': 2})


In [17]:
train_iter, test_iter = data.Iterator.splits((train, test), 
                                             batch_sizes=(5,5), 
                                             sort_key=lambda x: len(x.Text))

In [18]:
# check whether a batch is encoded and padded well
for it in train_iter:
    print(it)
    break
    
it.Text
it.Label


[torchtext.data.batch.Batch of size 5]
	[.Text]:('[torch.LongTensor of size 5x192]', '[torch.LongTensor of size 5]')
	[.Label]:[torch.LongTensor of size 5]


tensor([0, 0, 0, 1, 1])

In [19]:
it.Text

(tensor([[14862,   171,  5518,  1637,  5162, 12478, 21362, 17982, 16696,    14,
           4152,   541,  6172, 23714, 11344,    26,  2058, 12475, 17127,    81,
             52,  6852,  3110,  1874,   179,    44,  1110, 18592,  7248,  3472,
           7250,  3472,   702, 18593,  5751,   278,     2, 12213, 19042,  6029,
            265,     9,   828,   909,    34,     7,  1375,   199,  2380,     5,
           4883, 29278,  1905,    10, 10506, 24465,    13,   890, 21056,   258,
             64,  4636, 29397,   399,  1356,  3216,     5,  4885,  4739, 21096,
           1562,   115, 19178,  3720,  9936,  3525,  2840,    72,   202,  2942,
            530, 27093,   594,  3253,     7,   512,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,   

In [20]:
it.Label.shape,len(TEXT.vocab)

(torch.Size([5]), 29509)

In [21]:
# make a dictionary to decode 
dict_decode = {idx: keyword for keyword, idx in TEXT.vocab.stoi.items()}

In [22]:
dict_decode

{0: '<unk>',
 1: '<pad>',
 2: '너무',
 3: '좀',
 4: '것',
 5: '정말',
 6: '더',
 7: '수',
 8: '다',
 9: '잘',
 10: '맛이',
 11: '그냥',
 12: '진짜',
 13: '많이',
 14: '있는',
 15: '조금',
 16: '맛있는',
 17: '다른',
 18: '한',
 19: '엄청',
 20: '같이',
 21: '이',
 22: '좋은',
 23: '맛있게',
 24: '또',
 25: '안',
 26: '그',
 27: '맛은',
 28: '그리고',
 29: '맛',
 30: '여기',
 31: '먹고',
 32: '꼭',
 33: '아주',
 34: '먹을',
 35: '생각보다',
 36: '제일',
 37: '맛있어요',
 38: '있어서',
 39: '샐러드',
 40: '하지만',
 41: '맛도',
 42: '같은',
 43: '때',
 44: ':',
 45: '꽤',
 46: '약간',
 47: '않고',
 48: '근데',
 49: '딱',
 50: '양이',
 51: '먹었는데',
 52: '좋고',
 53: '살짝',
 54: '맛있다',
 55: '먹는',
 56: '분위기도',
 57: '맛있고',
 58: '역시',
 59: '곳',
 60: '먹으면',
 61: '가장',
 62: '그래도',
 63: '뭔가',
 64: '특히',
 65: 'ㅠㅠ',
 66: '나는',
 67: 'ㅎㅎ',
 68: '매우',
 69: '별로',
 70: '이렇게',
 71: '그런지',
 72: '분위기',
 73: '시켰는데',
 74: '없는',
 75: '참',
 76: '가서',
 77: '맛을',
 78: '저는',
 79: '함께',
 80: '굉장히',
 81: '고기',
 82: '맛있어서',
 83: '위에',
 84: '개인적으로',
 85: '건강한',
 86: '느낌',
 87: '완전',
 88: '있다.',
 89: '계속',
 9

In [23]:
(" ").join([dict_decode[idx.item()] for idx in it.Text[0][1]])

'#논현동 #항차이#다양성에 더욱 기대가 되는 중식당 대만식 중식당으로 요즘 많이 핫한 중식당 이에요 특히 돼지고기 튀김과 돼지고기 파볶음이 유명한 곳이지요 점심으로 방문해서 유명하다는 메뉴 두 가지를 먹어 봤습니다 #대만식돼지고기 마늘 튀김 바로 이 돼지고기 튀김으로 생활의 달인에 출연 하셨습니다 일명 소스없는 탕수육 이라고도 불리는 것 같아요 비주얼은 일일향의 육즙탕수육과 비슷합니다 촉촉하게 튀겨진 튀김옷은 잘 밑간된 돼지고기와 함께 향긋함을 줍니다 강하지는 않지만 간장과 마늘과 후추의 맛이 밑간에서 나는 것 같아요 튀김옷에도 어느정도 간이 되있어서 전체적으로 간간한 튀김이에요 맛은 상당히 뛰어난데.. 고기는 조금 퍽퍽한 느낌은 있습니다이 퍽퍽함을 함께 주시는 마늘간장이 해결해 줍니다 다진 마늘이 들어가 있는 매운 초간장 st의 소스인데... 튀김과 잘 어울립니다 퍽퍽한 고기에 스며들어 모이스쳐도 줍니다 참 맛있는 돼지고기 튀김이였어요 고량주 안주로 참 좋습니다 #대만식 돼지고기파볶음 무슨 맛일까 굉장히 궁금했던 메뉴에요 짭쫄 매콤... 밥과 함께 먹으니 밥도둑입니다 다진 돼지고기의 오돌돌 식감과 고추 파의 식감 또한 일정하고 정교하네요 혀에서 밥과 함께 잘 어우러지는 작은 알갱이 식감이 만족스럽습니다다만 볶음요리가 덜 볶아진 느낌이고 불향과 불맛 같은 불의 느낌이 굉장히 모자른 것 같아 의아하면서 아쉬웠습니다 #종합 보통 중식당에서 볼 수 없는 요리들과 면류들이 많아 자주 가보고 먹어보고 싶은 곳이네요 맛도 준수하고 서비스도 친절해서 가족식사고 연인 데이트도.. 회식으로도 참 좋을 것 같아요 여긴 꾸준히 가고 싶은 곳이네요'

*Bidirectional LSTM model*

In [34]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           batch_first=True,
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        #text = |batch size, sent len|
        
        #embedded = |batch size, sent len, emb dim|
        embedded = self.dropout(self.embedding(text))
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, torch.sort(text_lengths, descending=True)[0], batch_first=True)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = |batch size, sent len, hid dim * num directions|

        #hidden = |batch size, num layers * num directions, hid dim|        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout     
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
        return self.fc(hidden)

In [35]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [36]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [37]:
# adam optimizer with cross entropy loss
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

In [38]:
def categorical_accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True)
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [39]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.Text
        
        predictions = model(text, text_lengths)
        
        loss = criterion(predictions, batch.Label)
        
        acc = categorical_accuracy(predictions, batch.Label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [40]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            text, text_lengths = batch.Text
            predictions = model(text, text_lengths)
            
            loss = criterion(predictions, batch.Label)
            
            acc = categorical_accuracy(predictions, batch.Label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [41]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [42]:
N_EPOCHS = 5

best_valid_loss = float('inf')
y_hat_list= []
y = []
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, test_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 20s
	Train Loss: 0.659 | Train Acc: 79.09%
	 Val. Loss: 0.611 |  Val. Acc: 81.76%
Epoch: 02 | Epoch Time: 2m 18s
	Train Loss: 0.614 | Train Acc: 79.20%
	 Val. Loss: 0.609 |  Val. Acc: 81.53%
Epoch: 03 | Epoch Time: 2m 26s
	Train Loss: 0.588 | Train Acc: 79.46%
	 Val. Loss: 0.628 |  Val. Acc: 81.53%
Epoch: 04 | Epoch Time: 2m 28s
	Train Loss: 0.581 | Train Acc: 79.25%
	 Val. Loss: 0.623 |  Val. Acc: 81.76%
Epoch: 05 | Epoch Time: 2m 29s
	Train Loss: 0.486 | Train Acc: 80.56%
	 Val. Loss: 0.727 |  Val. Acc: 77.22%


In [181]:
criterion

CrossEntropyLoss()

In [179]:
# weights
model.state_dict()

OrderedDict([('embedding.weight',
              tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
                      [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
                      [-0.8514, -2.0202, -0.2351,  ...,  1.3333, -1.1639,  0.4074],
                      ...,
                      [ 1.2787, -0.8678,  0.0207,  ...,  0.3016,  0.1373,  0.3206],
                      [-0.4025,  1.6963, -0.1037,  ..., -0.0797, -0.8480,  0.5259],
                      [-0.3121,  1.8255,  1.4474,  ...,  0.2917, -1.9967, -0.0499]])),
             ('rnn.weight_ih_l0',
              tensor([[ 0.0495,  0.0160,  0.0030,  ...,  0.0901,  0.0407,  0.0376],
                      [ 0.0303,  0.0684, -0.0776,  ...,  0.1575, -0.1053, -0.0090],
                      [ 0.0607,  0.0345, -0.0538,  ...,  0.1387, -0.0146, -0.1341],
                      ...,
                      [-0.0448,  0.0811,  0.0600,  ..., -0.0076, -0.0042, -0.0595],
                      [ 0.0040,  0.